In [42]:
# From 10.1007/s10853-012-6710-x 
# Results were sent to me by email from Dr. Brillo (DLR)

# First column: Temperature (K), second column: 1/T * 10,000 (1/K), 
# third column: Viscosity (mPa.s), fourth column: log(viscosity) (mPa.s)
folder = "schick_experimental_results"
compositions = ["Al10Cu90", "Al20Cu80", "Al30Cu70", "Al40Cu60", "Al50Cu50", "Al50Cu50_II", "Al70Cu30", "Al80Cu20"]


In [43]:
# Read in the data

data = {}
for comp in compositions:
    filename = f"{folder}/{comp}.txt"
    with open(filename, "r") as f:
        lines = f.readlines()
        temps = []
        inv_temps = []
        viscosities = []
        log_viscosities = []
        for line in lines:
            parts = line.split()
            if '--' in parts:
                continue
            if comp == "Al80Cu20":
                # For Al80Cu20, viscosity is in the 4th column, log(viscosity) in the 3rd (error in original data file)
                temps.append(float(parts[0].replace(',', '.')))
                inv_temps.append(float(parts[1].replace(',', '.')))
                log_viscosities.append(float(parts[2].replace(',', '.')))
                viscosities.append(float(parts[3].replace(',', '.')))

            else:
                temps.append(float(parts[0].replace(',', '.')))
                inv_temps.append(float(parts[1].replace(',', '.')))
                viscosities.append(float(parts[2].replace(',', '.')))
                log_viscosities.append(float(parts[3].replace(',', '.')))

        data[comp] = {
            "Temperature (K)": temps,
            "1/T * 10,000 (1/K)": inv_temps,
            "Viscosity (mPa.s)": viscosities,
            "log(Viscosity) (mPa.s)": log_viscosities
        }


In [44]:
# Replace compositions 
new_compositions = ["0.90", "0.80", "0.70", "0.60", "0.50", "0.50_II", "0.30", "0.20"]
data_renamed = {}
for old_comp, new_comp in zip(compositions, new_compositions):
    data_renamed[new_comp] = data[old_comp]

In [45]:
from dfttk.plotly_format import plot_format
import plotly.graph_objects as go

# Plot viscosity vs temperature for all compositions
fig = go.Figure()
for comp in new_compositions:
    fig.add_trace(go.Scatter(
        x=data_renamed[comp]["Temperature (K)"],
        y=data_renamed[comp]["Viscosity (mPa.s)"],
        mode='lines+markers',
        name=f"x = {comp}"
    ))

plot_format(fig, "Temperature (K)", "Viscosity (mPa.s)")
fig.show()

In [49]:
import numpy as np

# Choose the temperatures you want to plot
select_temps = [1500, 1600, 1700, 1800]

# Only use the main compositions, ignore '0.50_II'
main_compositions = ["0.90", "0.80", "0.70", "0.60", "0.50", "0.30", "0.20"]
x_values = [0.90, 0.80, 0.70, 0.60, 0.50, 0.30, 0.20]

fig2 = go.Figure()

for target_temp in select_temps:
    x_vals = []
    visc_vals = []
    for comp, x in zip(main_compositions, x_values):

        temps = np.array(data_renamed[comp]["Temperature (K)"])
        viscosities = np.array(data_renamed[comp]["Viscosity (mPa.s)"])
        # Find the index of the temperature closest to target_temp
        idx = (np.abs(temps - target_temp)).argmin()
        x_vals.append(x)
        visc_vals.append(viscosities[idx])
    fig2.add_trace(go.Scatter(
        x=x_vals,
        y=visc_vals,
        mode='markers+lines',
        name=f"Viscosity at {target_temp} K"
    ))

# Simulation data TODO: Check this part
x_sim = [0, 0.25, 0.5, 0.75, 1, 0, 0.25, 0.5, 0.75, 1, 0, 0.25, 0.5, 0.75, 1, 0, 0.25, 0.5, 0.75, 1]
T_sim = [1500, 1500, 1500, 1500, 1500, 1600, 1600, 1600, 1600, 1600, 1700, 1700, 1700, 1700, 1700, 1800, 1800, 1800, 1800, 1800]
eta_sim = [1.06, 1.68, 2.70, 3.27, 3.86, 0.96, 1.44, 2.35, 3.07, 3.51, 0.88, 1.36, 2.03, 2.74, 2.30, 0.89, 1.28, 1.79, 2.33, 2.18]

for temp in select_temps:
    x_temp = [x for x, T in zip(x_sim, T_sim) if T == temp]
    eta_temp = [e for e, T in zip(eta_sim, T_sim) if T == temp]
    fig2.add_trace(go.Scatter(
        x=x_temp,
        y=eta_temp,
        mode='markers+lines',
        name=f"Viscosity at {temp} K (sim)",
        marker=dict(symbol='diamond')
    ))

plot_format(fig2, "x (Cu fraction)", "Viscosity (mPa.s)")
fig2.show()

In [ ]:
x = [0, 0.25, 0.5, 0.75, 1, 0, 0.25, 0.5, 0.75, 1, 0, 0.25, 0.5, 0.75, 1, 0, 0.25, 0.5, 0.75, 1]
T = [1500, 1500, 1500, 1500, 1500, 1600, 1600, 1600, 1600, 1600, 1700, 1700, 1700, 1700, 1700, 1800, 1800, 1800, 1800, 1800]
eta = [1.06, 1.68, 2.70, 3.27, 3.86, 0.96, 1.44, 2.35, 3.07, 3.51, 0.88, 1.36, 2.03, 2.74, 2.30, 0.89, 1.28, 1.79, 2.33, 2.18]


12.5